# Analyse BTC daily features

Notebook de démonstration pour:
- vérifier la qualité des données BTC (btc_features_daily.csv)
- créer une cible (close du lendemain)
- explorer la corrélation des features
- entraîner quelques modèles vus en cours (baseline de persistance, régression linéaire/Ridge, Gradient Boosting, Random Forest)
- analyser les résultats et discuter des limites


## 1. Chargement des données
On charge le fichier `btc_features_daily.csv`, on trie par date et on inspecte le taux de valeurs manquantes.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

data_path = Path('btc_features_daily.csv')
df = pd.read_csv(data_path, parse_dates=['timestamp']).sort_values('timestamp').reset_index(drop=True)
print(f"Loaded {len(df):,} rows, {len(df.columns)} columns")
missing = df.isna().mean().sort_values(ascending=False)
print('Taux de valeurs manquantes (top 10):')
print(missing.head(10))

df.head()


Loaded 3,973 rows, 52 columns
Taux de valeurs manquantes (top 10):
timestamp      0.0
adj close      0.0
stochrsi_k     0.0
stochrsi_d     0.0
macd           0.0
macd_signal    0.0
macd_hist      0.0
trix_14        0.0
trix_signal    0.0
atr_14         0.0
dtype: float64


,timestamp,adj close,close,high,low,open,volume,ret_1,log_ret,ret_5,...,log_vol_chg,cci_20,plus_di_14,minus_di_14,adx_14,roll_skew_20,roll_kurt_20,typ_price,median_price,ohlc4
0,2014-12-24,322.533997,322.533997,334.740997,321.356995,334.385010,15092300,-0.035980,-0.036644,0.014759,...,-0.093663,-55.910258,16.714946,30.030404,25.159419,0.074377,-0.783043,326.210663,328.048996,328.254250
1,2014-12-25,319.007996,319.007996,322.670013,316.958008,322.286011,9883640,-0.010932,-0.010992,-0.033180,...,-0.423304,-65.674105,16.168300,31.566923,25.666490,0.216627,-0.483404,319.545339,319.814011,320.230507
2,2014-12-26,327.924011,327.924011,331.424011,316.627014,319.152008,16410500,0.027949,0.027566,0.022070,...,0.507041,-38.540179,19.762835,28.927653,25.177641,0.130736,-0.852232,325.325012,324.025513,323.781761
3,2014-12-27,315.863007,315.863007,328.911011,312.630005,327.583008,15185200,-0.036780,-0.037473,-0.048279,...,-0.077600,-47.473235,17.981409,28.533062,24.999572,0.267018,-0.988402,319.134674,320.770508,321.246758
4,2014-12-28,317.239014,317.239014,320.028015,311.078003,316.160004,11676600,0.004356,0.004347,-0.051806,...,-0.262734,-47.802507,17.070467,27.966059,24.941944,0.103988,-0.892805,316.115011,315.553009,316.126259


## 2. Cible et split temporel
On construit la cible `target_close_next` = close du lendemain (shift -1), puis split chrono (70% train, 15% val, 15% test) pour éviter le look-ahead.


In [ ]:
# Créer la cible
df = df.copy()
df['target_close_next'] = df['close'].shift(-1)
df = df.iloc[:-1]

feature_cols = [c for c in df.columns if c not in ['timestamp', 'target_close_next']]
X = df[feature_cols]
y = df['target_close_next']

n = len(df)
train_end = int(n * 0.7)
val_end = int(n * 0.85)
X_train, y_train = X.iloc[:train_end], y.iloc[:train_end]
X_val, y_val = X.iloc[train_end:val_end], y.iloc[train_end:val_end]
X_test, y_test = X.iloc[val_end:], y.iloc[val_end:]
print(f"Split sizes -> train: {len(X_train)}, val: {len(X_val)}, test: {len(X_test)}")

y.describe()


Split sizes -> train: 2780, val: 596, test: 596


count      3972.000000
mean      26761.019587
std       31230.206548
min         178.102997
25%        3611.820068
50%       10972.544922
75%       41980.804688
max      124752.531250
Name: target_close_next, dtype: float64

## 3. Corrélation avec la cible
Les features prix (close, adj close, typ_price, etc.) sont très corrélées au close du lendemain (>0.99), signalant une forte redondance.


In [ ]:
corr = (
    df.drop(columns=['timestamp'])
      .corr()['target_close_next']
      .drop('target_close_next')
      .abs()
      .sort_values(ascending=False)
)
print('Top 10 corrélations (absolues):')
print(corr.head(10))


Top 10 corrélations (absolues):
adj close       0.999377
close           0.999377
typ_price       0.999328
ohlc4           0.999259
median_price    0.999248
high            0.999135
low             0.999077
open            0.998810
wma_10          0.998472
ma_7            0.998334
Name: target_close_next, dtype: float64


## 4. Modèles testés
On teste :
- Baseline persistance : prédire que le close de demain = close d’aujourd’hui.
- Régression linéaire + scaling.
- Ridge (régularisation L2).
- Gradient Boosting Regressor.
- Random Forest Regressor.

On évalue sur le set de test (split chrono) avec MAE, RMSE, MAPE.



In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

def rmse(y_true, y_pred):
    return float(mean_squared_error(y_true, y_pred) ** 0.5)

def mape(y_true, y_pred):
    pct_err = np.abs((y_true - y_pred) / y_true)
    pct_err = pct_err.replace([np.inf, -np.inf], np.nan).dropna()
    return float(pct_err.mean() * 100)

def evaluate(name, model):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    return {
        'model': name,
        'MAE': mean_absolute_error(y_test, pred),
        'RMSE': rmse(y_test, pred),
        'MAPE%': mape(y_test, pred),
    }

results = []

# Baseline persistance
persistence_pred = X_test['close']
results.append({
    'model': 'persistence_close',
    'MAE': mean_absolute_error(y_test, persistence_pred),
    'RMSE': rmse(y_test, persistence_pred),
    'MAPE%': mape(y_test, persistence_pred),
})

models = [
    make_pipeline(StandardScaler(), LinearRegression()),
    make_pipeline(StandardScaler(), Ridge(alpha=1.0)),
    GradientBoostingRegressor(random_state=0),
    RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1, min_samples_leaf=2),
]
names = ['linear_reg', 'ridge', 'gboost', 'rf']

for name, model in zip(names, models):
    results.append(evaluate(name, model))

results = sorted(results, key=lambda r: r['RMSE'])
results


[{'model': 'persistence_close',
  'MAE': 1476.206428271812,
  'RMSE': 2027.4586170587938,
  'MAPE%': 1.7421122546524472},
 {'model': 'linear_reg',
  'MAE': 1746.6667208275267,
  'RMSE': 2289.236441680717,
  'MAPE%': 2.015755013653118},
 {'model': 'ridge',
  'MAE': 1745.9527526736485,
  'RMSE': 2289.6992479917885,
  'MAPE%': 2.01568592331959},
 {'model': 'rf',
  'MAE': 25164.049599892835,
  'RMSE': 31994.987486438175,
  'MAPE%': 24.62397069216737},
 {'model': 'gboost',
  'MAE': 27485.749910038478,
  'RMSE': 33835.03767818492,
  'MAPE%': 27.394445643101843}]


### Explications des métriques utilisées
- **MAE (Mean Absolute Error)** : erreur absolue moyenne, exprimée en unités de prix (≈ dollars). Plus petit = mieux.
- **RMSE (Root Mean Squared Error)** : racine de l’erreur quadratique moyenne, pénalise davantage les grosses erreurs. Plus petit = mieux.
- **MAPE (Mean Absolute Percentage Error)** : erreur absolue moyenne en pourcentage du prix réel. Plus petit = mieux, lecture directe en “% d’erreur typique”.

### Lecture des résultats obtenus
- La baseline `persistence_close` (close de demain = close d’aujourd’hui) est la meilleure : MAE ~1.5k, RMSE ~2.0k, MAPE ~1.7%.
- `linear_reg` et `ridge` font légèrement pire : pas de gain vs recopier le prix du jour.
- `rf` et `gboost` sont beaucoup moins bons (erreurs 10x) : ils sur-apprennent ou n’ont pas de signal utile.

### Ce que cela implique
- Le dataset est dominé par l’autocorrélation du prix : le meilleur prédicteur du prix de demain reste le prix d’aujourd’hui.
- Les features sont très redondantes (corrélations >0.99), donc peu d’information nouvelle à extraire.
- Pour progresser :
  - Formuler la cible en rendement ou direction (up/down) plutôt qu’en niveau de prix.
  - Ajouter des signaux orthogonaux (lags de retours, volatilité, sentiment, on-chain, macro) et les aligner temporellement.
  - Utiliser une validation temporelle stricte (rolling) et plus de régularisation/contraintes pour limiter l’overfit des modèles complexes.

## 5. Importance des features (Random Forest)
Pour illustrer : importances des 10 principales features selon la Random Forest.

In [ ]:
rf = RandomForestRegressor(
    n_estimators=200, random_state=0, n_jobs=-1, min_samples_leaf=2
).fit(X_train, y_train)
importances = (
    pd.Series(rf.feature_importances_, index=feature_cols)
      .sort_values(ascending=False)
      .head(10)
)
importances


adj close       0.289137
typ_price       0.240868
close           0.226125
high            0.072490
median_price    0.065428
ohlc4           0.056057
low             0.032645
open            0.014368
obv             0.000455
wma_10          0.000162
dtype: float64

Random Forest identifie surtout les niveaux de prix actuels (adj close, typ_price, close) comme prédicteurs, signe d’une forte redondance des features. Malgré sa capacité à modéliser des non-linéarités, le modèle n’apporte pas de gain : les erreurs sont bien pires que la baseline, indiquant un sur-apprentissage ou un manque de signaux exogènes. Pour améliorer : mieux contraindre le modèle, changer la cible vers des rendements/direction, et enrichir les données avec des variables orthogonales (sentiment, on-chain, macro) en validation temporelle stricte.


## 6. Lecture des résultats
- La baseline de persistance (~MAE 1.48k, RMSE ~2.03k, MAPE ~1.74%) est la meilleure. Les modèles linéaires font légèrement pire. Les arbres/boosting explosent en erreur (MAPE > 20%).
- Le dataset est ultra redondant et dominé par le niveau de prix : les corrélations >0.99 montrent que prédire le prix de demain revient surtout à recopier celui d’aujourd’hui.
- Les modèles non contraints sur-apprennent sans gain réel. Pour améliorer :
  - Travailler sur des retours ou la direction (classification up/down) plutôt que sur le niveau de prix.
  - Ajouter des features réellement orthogonales (volatilité, lags, sentiment, on-chain, macro alignés temporellement).
  - Utiliser une validation temporelle (rolling) et plus de régularisation / features réduites (Ridge/ElasticNet, PCA sur indicateurs).
